In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [3]:
poblacion_teorica = pd.read_excel("../data/processed/proyeccion_problemas_de_salud.xlsx")

poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]
poblacion_teorica = poblacion_teorica.query("Estrato == 'Pais'")

DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()
poblacion_teorica = poblacion_teorica.set_index("Diagnostico")
# Selecciona solamente la poblacion prevista
poblacion_teorica = poblacion_teorica[[i for i in range(2017, 2036)]]

In [4]:
egresos_int = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_int = egresos_int[egresos_int["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_int = egresos_int.query("ANO_EGRESO >= 2017")

In [5]:
egresos_pais = pd.read_csv(
    "../data/processed/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
    usecols=[
        "ANO_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "DIAG1",
        "n_egresos",
        "dias_estada_totales",
        "n_int_q",
        "n_muertos",
    ],
)

egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]
egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Separación de población teórica en hospitalizados y ambulatorios

In [6]:
# De los casos totales, se asume que una parte (0.8) se resuelve en hospitalizacion
RATIO_HOSPITALIZADOS = 0.8
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

poblacion_teorica_hospitalizados = poblacion_teorica * RATIO_HOSPITALIZADOS
poblacion_teorica_ambulatorio = poblacion_teorica * RATIO_AMBULATORIO

In [7]:
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
)

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,
    index="DIAG1",
    columns="ANO_EGRESO",
    values=["n_egresos"],
    aggfunc="sum",
    fill_value=0,
)

In [8]:
tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

## Obtención de brecha de atención del país

En este apartado se quiere saber si el país ha suplido la necesidad de atención para cada problema
de salud. Para esto, se contrastará la cantidad de casos atendidos a nivel país en una hospitalización con
los casos teóricos según la incidencia. La fórmula será:

$ Brecha = \frac{Casos Atendidos Reales - Casos Teoricos}{Casos Teoricos}$

En el caso que la brecha sea positiva, entonces $Casos Atendidos Reales > Casos Teoricos$, y el pais
ha podido suplir la demanda sanitaria del problema de salud. En caso contrario, el país ha sido
incapaz de suplir la demanada para resolver estos problemas de salud. Además, se puede saber
el porcentaje de los casos teóricos que ha suplido el país. Por ejemplo:

$Brecha = 1.12$ Indica que el país ha resolvido todos los casos teóricos del problema de salud.
Además, ha resuelto un 12% más que los casos teóricos.

In [32]:
brecha_pais = (
    tabla_dinamica_egresos_pais - poblacion_teorica_hospitalizados.round(0)
) / poblacion_teorica_hospitalizados.round(0)

In [34]:
display(brecha_pais)

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
C33X,0.666667,0.200000,0.133333,-0.125000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C340,-0.839725,-0.861836,-0.859716,-0.904097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C341,-0.634218,-0.538164,-0.532701,-0.705773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C342,-0.950836,-0.934300,-0.950711,-0.955307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C343,-0.746313,-0.716908,-0.735545,-0.792365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C381,-0.768362,-0.772222,-0.715847,-0.775401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C384,6.800000,7.800000,9.400000,7.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C450,15.200000,14.200000,14.000000,9.600000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C780,-0.633235,-0.633816,-0.592417,-0.696462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C782,-0.855457,-0.818357,-0.800000,-0.813780,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
